In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness

In [3]:
from constant.bi.question import QA_PAIRS as QA_PAIRS_BI
from constant.ojk.question_ojk import EVAL_RESULTS
from constant.sikepo.eval import EVAL_QUESTIONS as q_sikepo
from constant.sikepo.eval import EVAL_ANSWERS as gt_sikepo

q_bi = []
gt_bi = []
for k, v in QA_PAIRS_BI.items():
    for key, val in v.items():
        q_bi.append(val['question'])
        gt_bi.append(val['answer'])

q_ojk = []
gt_ojk = []
from constant.ojk.question_ojk import EVAL_RESULTS
for k, v in EVAL_RESULTS.items():
    q_ojk.append(v['question'])
    gt_ojk.append(v['answer'])

In [4]:
questions = q_ojk + q_bi + q_sikepo
ground_truth = gt_ojk + gt_bi + gt_sikepo

In [5]:
print(ground_truth)

['SWDKLLJ merupakan premi asuransi yang dibayarkan oleh para pengusaha/pemilik alat angkutan lalu lintas jalan kepada perusahaan yang menyelenggarakan Dana Kecelakaan Lalu Lintas Jalan.\n\nSource: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=2)', 'Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris sesuai dengan ketentuan dalam Peraturan Nomor III.A.4 tentang Tata Cara Penyusunan serta Pengajuan Rencana Anggaran dan Penggunaan Laba Bursa Efek (KEP-05/PM/1996).\n\nSource: KEP-05/PM/1996 (https://www.ojk.go.id/id/regulasi/Documents/Pages/BAPEPAM-tentang-Tata-Cara-Penyusunan-serta-Pengajuan-Rencana-Anggaran-dan-Penggunaan-Laba-Bursa-Efek/IIIA4_Tata%20Cara%20Penyusunan-Pengajuan%20Rencana%20Anggaran%20dan%20Penggunaan%20Laba%20BE.pdf)', 'Keputusan Ketua Badan Pengawas Pasar Modal Nomor KEP-

In [6]:
questions[1]

'Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris dengan ketentuan apa?'

In [7]:
print(len(questions))
print(len(ground_truth))


133
133


In [20]:
from main import chain, chain_wo_self

In [7]:
# answer = []
# context = []

In [8]:
# from lark import UnexpectedCharacters
# from langchain_core.exceptions import OutputParserException
# i = 1

# for question in questions:
#     try:
#         result = chain.invoke({"question":question, "chat_history":"-"})
#         print("Successfully run the chain for question", i)
#         answer.append(result['answer'])
#         if 'context_text' in result:
#             context.append(result['context_text'])
#         else:
#             context.append(result['context'])
#         i += 1

#     except OutputParserException as e:
#         result = chain_wo_self.invoke({"question":question, "chat_history":"-"})
#         print("Successfully run the chain_wo_self for question", i)
#         answer.append(result['answer'])
#         if 'context_text' in result:
#             context.append(result['context_text'])
#         else:
#             context.append(result['context'])

#         i += 1
    
#     except Exception:
#         print("Failed run the chain for question", i)
#         break


In [8]:
import pickle

with open("answer_eval.pkl", 'rb') as file:
    answer = pickle.load(file)

with open("context_eval.pkl", 'rb') as file:
    context = pickle.load(file)


In [9]:
print(len(context))

133


In [23]:
from lark import UnexpectedCharacters
from langchain_core.exceptions import OutputParserException

i = 82
selected_questions = questions[i-1:]

for question in selected_questions:
    try:
        result = chain.invoke({"question":question, "chat_history":"-"})
        print("Successfully run the chain for question", i)
        answer.append(result['answer'])
        if 'context_text' in result:
            context.append(result['context_text'])
        else:
            context.append(result['context'])
        i += 1

    except OutputParserException as e:
        result = chain_wo_self.invoke({"question":question, "chat_history":"-"})
        print("Successfully run the chain_wo_self for question", i)
        answer.append(result['answer'])
        if 'context_text' in result:
            context.append(result['context_text'])
        else:
            context.append(result['context'])

        i += 1
    
    except Exception:
        print("Failed run the chain for question", i)
        break




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Peraturan {nomor_ketentuan: '17/2/PBI/2015', jenis_bank: 'Bank Indonesia', jenis_ketentuan: 'Suku Bunga Penawaran Antarbank'})
RETURN p.tanggal_berlaku AS tanggal_berlaku

Full Context:
[]

> Finished chain.
Successfully run the chain for question 82
Successfully run the chain for question 83
Successfully run the chain for question 84


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p1:Peraturan {jenis_ketentuan: 'Undang-Undang', nomor_ketentuan: '10 Tahun 1998'})-[:DIUBAH]->(p2:Peraturan {jenis_bank: 'Perbankan'})
RETURN p2.nomor_ketentuan AS nomor_undang_undang

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p1:Peraturan {jenis_ketentuan: 'Undang-Undang', nomor_ketentuan: '10 Tahun 1998'})-[:DIUBAH]->(p2:Peraturan {jenis_bank: 'Perbankan'})
RETURN p2.nomor_ketentuan AS nomor_undang_undang

Full Context:
[]

> Fini

In [8]:
len(ground_truth)

133

In [10]:
import pickle

with open("answer_eval.pkl", 'wb') as file:
    pickle.dump(answer, file)

with open("context_eval.pkl", 'wb') as file:
    pickle.dump(context, file)


In [9]:
question

'Peraturan Bank Indonesia Nomor 7/33/PBI/2005 berisi pencabutan atas peraturan bank indonesia nomor berapa dan tentang apa?'

In [13]:
question

'Siapa yang menandatangani keputusan ketua bapepam nomor kep-07/PM/1996?'

In [12]:
'context_text' in result

True

In [12]:
len(context)

4

In [13]:
len(answer)

4

In [14]:
i

4

## Ragas

In [10]:
new_context = context.copy()

for i in range(len(context)):
    if type(context[i]) is not list:
        new_context[i] = [context[i].text]


In [11]:
from datasets import Dataset

dict_eval = {
    "question": questions,
    "answer": answer,
    "contexts": new_context,
    "ground_truth": ground_truth
}

dataset = Dataset.from_dict(dict_eval)

In [13]:
print(questions)

['Apa itu SWDKLLJ?', 'Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris dengan ketentuan apa?', 'KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOMOR KEP-06/PM/1996 tentang apa?', 'Penolakan atas permohonan persetujuan mengenai pengajuan atau perubahan peraturan Lembaga Kliring dan Penjaminan dilakukan dengan menggunakan formulir nomor berapa?', 'Anggota Dewan Pengawas Syariah Perusahaan Pembiayaan dilarang untuk apa?', 'Berapa uang pertanggungan dan pertanggungan total loss only (tlo) untuk kategori 5?', 'Perusahaan wajib menyampaikan laporan hasil pengawasan Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 2 dalam bentuk apa, kepada siapa, dan paling lambat kapan?', 'Batas tingkat solvabilitas minimum bagi perusahaan asuransi dan perusahaan reasuransi ditingkatkan berdasarkan apa?', 'Kapan PERATURAN KETUA BADAN PENGAWAS PASAR MODAL DAN LEMBAGA KEUANGAN NOMOR: PER- 10/BL/2012 mulai berlaku?', 'Apa itu Dana Tabarru’', 'Dalam rangka mendo

In [14]:
range(4)

range(0, 4)

In [13]:
first_third = dataset.select(range(0, 44))
first_third

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 44
})

In [16]:
from utils.config import get_config
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

config = get_config()

# =========== GLOBAL VARIABLES ===========
llm_model, embed_model = get_model(model_name=ModelName.OPENAI, config=config,
                                   llm_model_name=LLMModelName.GPT_4O_MINI, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

In [15]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

first_score = evaluate(first_third, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)

Evaluating:   0%|          | 0/308 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.


In [21]:
first_df = first_score.to_pandas()
first_df.to_csv("ragas_eval_q1_to_q44.csv")


In [26]:
import pandas as pd

first_df = pd.read_csv("ragas_eval_q1_to_q44.csv")
second_df = pd.read_csv("ragas_eval_q44_to_q66.csv")

In [24]:
second_third = dataset.select(range(44, 66))
second_score = evaluate(second_third, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)
second_df = second_score.to_pandas()
second_df.to_csv("ragas_eval_q44_to_q66.csv")

Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

In [17]:
third = dataset.select(range(66, 88))
third_score = evaluate(third, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)
third_df = third_score.to_pandas()
third_df.to_csv("ragas_eval_q66_to_q88.csv")

Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

In [18]:
fourth = dataset.select(range(88, 110))
fourth_score = evaluate(fourth, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)
fourth_df = fourth_score.to_pandas()
fourth_df.to_csv("ragas_eval_q88_to_q110.csv")

Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.


In [23]:
fifth = dataset.select(range(110, 133))
fifth_score = evaluate(fifth, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)
fifth_df = fifth_score.to_pandas()
fifth_df.to_csv("ragas_eval_q110_to_q133.csv")

Evaluating:   0%|          | 0/161 [00:00<?, ?it/s]

Failed to parse output. Returning None.


In [44]:
df = pd.concat([first_df, second_df, third_df, fourth_df, fifth_df ])
df.head()

,Unnamed: 0,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,answer_similarity,answer_correctness,harmfulness
0,0.0,Apa itu SWDKLLJ?,SWDKLLJ adalah singkatan dari Sistem Wajib Den...,['Sistem Wajib Denda Kecelakaan Lalu Lintas Ja...,SWDKLLJ merupakan premi asuransi yang dibayark...,0.666667,0.000000,0.000000,1.0,0.626631,0.585229,0
1,1.0,Bursa Efek wajib menyampaikan laporan realisas...,Berdasarkan informasi yang diperoleh dari sumb...,['Bursa Efek wajib menyampaikan laporan realis...,Bursa Efek wajib menyampaikan laporan realisas...,0.571429,0.000000,1.000000,1.0,0.899541,0.599892,0
2,2.0,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,['KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL N...,Keputusan Ketua Badan Pengawas Pasar Modal Nom...,0.625000,0.000000,0.500000,0.0,0.721185,0.180296,0
3,3.0,Penolakan atas permohonan persetujuan mengenai...,Dalam pengajuan atau perubahan peraturan Lemba...,['Formulir nomor yang digunakan untuk menolak ...,Penolakan atas permohonan persetujuan mengenai...,0.500000,0.713170,1.000000,1.0,0.882475,0.720619,0
4,4.0,Anggota Dewan Pengawas Syariah Perusahaan Pemb...,Larangan bagi anggota Dewan Pengawas Syariah P...,['Larangan bagi anggota Dewan Pengawas Syariah...,Anggota Dewan Pengawas Syariah Perusahaan Pemb...,1.000000,0.725574,0.833333,1.0,0.820382,0.441938,0


In [45]:
df = df.drop("Unnamed: 0", axis=1)

In [ ]:
df = df.reset_index(drop=True)
df

In [47]:
df.to_csv("ragas_eval_result.csv")

In [ ]:
df.describe()

## Result

In [38]:
import pandas as pd

ojk_df = pd.read_excel("ojk_eval_new.xlsx")
bi_df = pd.read_excel("bi_eval.xlsx")
sikepo_df = pd.read_excel("sikepo_eval.xlsx")

df = pd.concat([ojk_df, bi_df, sikepo_df])
df = df.drop("Unnamed: 0", axis=1)
df = df.reset_index(drop=True)
df.head()

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,answer_similarity,answer_correctness,harmfulness
0,Apa itu SWDKLLJ?,SWDKLLJ adalah singkatan dari Sistem Wajib Den...,['Sistem Wajib Denda Kecelakaan Lalu Lintas Ja...,Sumbangan Wajib Dana Kecelakaan Lalu Lintas ad...,0.666667,0.850381,0.0,0.0,0.536681,0.348389,0
1,Bursa Efek wajib menyampaikan laporan realisas...,Berdasarkan informasi yang diperoleh dari sumb...,['Bursa Efek wajib menyampaikan laporan realis...,Bursa Efek wajib menyampaikan laporan realisas...,0.571429,0.748256,1.0,0.0,0.757334,0.376834,0
2,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,['KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL N...,TATA CARA PEMBERIAN PERSETUJUAN ANGGARAN DASAR...,1.000000,0.481839,0.0,0.0,0.482538,0.270635,0
3,Penolakan atas permohonan persetujuan mengenai...,Dalam pengajuan atau perubahan peraturan Lemba...,['Formulir nomor yang digunakan untuk menolak ...,Penolakan atas permohonan persetujuan mengenai...,0.500000,0.596502,1.0,1.0,0.845767,0.511442,0
4,Anggota Dewan Pengawas Syariah Perusahaan Pemb...,Larangan bagi anggota Dewan Pengawas Syariah P...,['Larangan bagi anggota Dewan Pengawas Syariah...,Dilarang melakukan rangkap jabatan sebagai ang...,1.000000,0.725574,1.0,1.0,0.776191,0.430890,0


In [39]:
df.describe()

,faithfulness,answer_relevancy,context_precision,context_recall,answer_similarity,answer_correctness,harmfulness
count,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000
mean,0.736581,0.737722,0.843458,0.657543,0.660813,0.520065,0.056075
std,0.218890,0.164484,0.306872,0.446711,0.170311,0.226867,0.231149
min,0.166667,0.291350,0.000000,0.000000,0.304411,0.092186,0.000000
25%,0.571429,0.646608,0.833333,0.000000,0.511434,0.348249,0.000000
50%,0.800000,0.749658,1.000000,1.000000,0.694352,0.468216,0.000000
75%,0.916084,0.854546,1.000000,1.000000,0.806336,0.673710,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.924212,0.958881,1.000000


## Test

In [6]:
result = chain.invoke({"question":"siapa yang bisa memiliki bank", "chat_history":"-"})

In [7]:
result

{'rewrited question': 'Siapa yang dapat memiliki bank?',
 'context': [Document(metadata={'doc_id': 2291, 'title': 'Surat Edaran Bank Indonesia Nomor 7/14/DPNP', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '7/14/DPNP/2005', 'effective_date': '2005/04/18', 'file_url': 'https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2005/lamp_se_071405.pdf', 'page_number': 14}, page_content='Bank A\nBank A\nBank A\nBank A \n \n \n \n(Repo Party)\n(Repo Party)\n(Repo Party)\n(Repo Party) \n \n \n \nPenerbit SSB\nPenerbit SSB\nPenerbit SSB\nPenerbit SSB \n \n \n \n(issuer)\n(issuer)\n(issuer)\n(issuer) \n \n \n \nRepo\nBMPK \nReverse Party \nBMPK \nRepo Party \n2\n\nSource: [7/14/DPNP/2005](https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2005/lamp_se_071405.pdf#page=14)'),
  Document(metadata={'doc_id': 2009, 'title': 'Surat Edaran Bank Indonesia Nomor 15/4/DPNP', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '15/4/DPNP/2013'

In [9]:
result['context_text']

['Pihak-pihak yang dapat memiliki bank adalah pihak-pihak yang memenuhi persyaratan yang ditetapkan oleh Bank Indonesia, termasuk memiliki akhlak dan moral yang baik, komitmen untuk mematuhi peraturan perundang-undangan yang berlaku, memiliki komitmen yang tinggi terhadap pengembangan operasional bank yang sehat, dan tidak termasuk dalam daftar orang-orang yang dilarang menjadi pemegang saham dan/atau pengurus bank sesuai dengan ketentuan yang ditetapkan oleh Bank Indonesia.\n\nReference:\n11/1/PBI/2009 (https://www.ojk.go.id/id/regulasi/Documents/Pages/PBI-tentang-Bank-Umum/153.pdf#page=18)\n6/24/PBI/2004 (https://www.ojk.go.id/id/kanal/perbankan/regulasi/peraturan-bank-indonesia/Documents/317.pdf#page=19)\n7 Tahun 1992 (https://www.ojk.go.id/id/regulasi/Documents/Pages/UU-Republik-Indonesia-tentang-Perbankan/UU%20Nomor%207%20Tahun%201992.pdf#page=45)',
 'Pihak-pihak yang dapat memiliki bank adalah warga negara Indonesia dan/atau badan hukum Indonesia, warga negara Indonesia dan/atau 